In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PROJECT_PATH = "/content/drive/MyDrive/cough_detection"

ZIP_PATH = PROJECT_PATH + "/Multimodal_Cough_Dataset.zip"

EXTRACT_PATH = PROJECT_PATH + "/dataset"

In [4]:
import zipfile

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_PATH)

print("Extraction complete")

Extraction complete


In [15]:
import os

os.listdir(EXTRACT_PATH)
DATASET_PATH = EXTRACT_PATH + "/Multimodal Cough Dataset"

os.listdir(DATASET_PATH)

['005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 'DataAnnotation.json']

In [16]:
!pip install librosa soundfile

In [17]:
import librosa
import numpy as np
import json
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [18]:
ANNOTATION_PATH = DATASET_PATH + "/DataAnnotation.json"

with open(ANNOTATION_PATH) as f:
    annotations = json.load(f)

print(len(annotations))

39


In [24]:
cough_dict = {}

for record in annotations:

    filename = record["original_filename"].lower()

    cough_dict[filename] = []

    for seg in record["segmentations"]:

        if "Cough" in seg["annotations"]:

            cough_dict[filename].append(
                (seg["start_time"], seg["end_time"])
            )

print("Total annotated files:", len(cough_dict))

example = list(cough_dict.keys())[0]

print(example, cough_dict[example][:3])

Total annotated files: 39
005_no_talking_in.wav [(228.016, 228.366), (399.856, 400.156), (568.497, 568.893)]


In [26]:
X = []
y = []

sr_target = 750

for participant in os.listdir(DATASET_PATH):

    participant_path = os.path.join(DATASET_PATH, participant)

    if not os.path.isdir(participant_path):
        continue


    for trial_folder in os.listdir(participant_path):

        trial_path = os.path.join(participant_path, trial_folder)


        # FIX: Skip files like sync_time.txt
        if not os.path.isdir(trial_path):
            continue


        for file in os.listdir(trial_path):

            if not file.endswith(".wav"):
                continue


            actual_filename = file.lower()

            if actual_filename not in cough_dict:
                continue


            audio_path = os.path.join(trial_path, file)

            audio, sr = librosa.load(audio_path, sr=sr_target)

            cough_times = cough_dict[actual_filename]


            window_size = int(2 * sr)
            step_size = int(0.5 * sr)


            for start in range(0, len(audio)-window_size, step_size):

                start_sec = start / sr
                end_sec = (start+window_size) / sr


                label = 0

                for cough_start, cough_end in cough_times:

                    if not (end_sec < cough_start or start_sec > cough_end):

                        label = 1
                        break


                mel = librosa.feature.melspectrogram(

                    y=audio[start:start+window_size],
                    sr=sr,
                    n_mels=64,
                    n_fft=256,
                    hop_length=64
                )

                log_mel = librosa.power_to_db(mel)

                X.append(log_mel.flatten())
                y.append(label)


print("Total samples:", len(X))

print("Cough samples:", sum(y))

Total samples: 57566
Cough samples: 10231


In [27]:
import numpy as np

X = np.array(X)
y = np.array(y)

print("Feature shape:", X.shape)
print("Label shape:", y.shape)

Feature shape: (57566, 1536)
Label shape: (57566,)


Split training and testing

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(

    X, y,

    test_size=0.2,

    stratify=y,

    random_state=42

)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

Training samples: 46052
Testing samples: 11514


Feature Scaling

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

Training

In [30]:
from sklearn.linear_model import LogisticRegression

baseline_model = LogisticRegression(

    max_iter=1000,

    C=0.1,

    random_state=42

)

baseline_model.fit(X_train, y_train)

print("Baseline model training completed")

Baseline model training completed


Testing

In [31]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# Predict labels
y_pred = baseline_model.predict(X_test)

# Predict probabilities
y_prob = baseline_model.predict_proba(X_test)[:,1]


# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate AUC
auc = roc_auc_score(y_test, y_prob)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)


print("Baseline Accuracy:", accuracy)

print("Baseline AUC:", auc)

print("\nConfusion Matrix:")
print(cm)

Baseline Accuracy: 0.8534827166927219
Baseline AUC: 0.8655578950715711

Confusion Matrix:
[[8971  497]
 [1190  856]]


Saving to Drive

In [32]:
import pickle

# Save baseline model
pickle.dump(
    baseline_model,
    open(PROJECT_PATH + "/baseline_model.pkl", "wb")
)

# Save scaler
pickle.dump(
    scaler,
    open(PROJECT_PATH + "/baseline_scaler.pkl", "wb")
)

print("Baseline model and scaler saved to Google Drive")

Baseline model and scaler saved to Google Drive
